Universidad del Valle de Guatemala

Departamento de Ciencias de la Computación

Inteligencia Artificial



Nadissa Vela - 23764

Cristian Túnchez - 231359

---


## Laboratorio 3

## Task 2




### Pre-procesamiento

In [ ]:
import pandas as pd
import re
import numpy as np

# a) Importamos el dataset
df = pd.read_table("/content/entrenamiento.txt", sep="\t", names=["label", "message"])

print("\nPrimeras filas del dataset:")
print(df.head())


Primeras filas del dataset:
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [ ]:
# b) Pasar todo a minúsculas, eliminar tildes y caracteres especiales
# implementar bag of words
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    return tokens

df["tokens"] = df["message"].apply(clean_text)
print("\n\nDataset limpio:")
df[["label", "tokens"]].head()



Dataset limpio:


,label,tokens
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, a, wkly, comp, to, win, fa, ..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [ ]:
# separar 80/20 la data

df_shuffled = df.sample(frac=1, random_state=42)
split_index = int(0.8 * len(df_shuffled))
train_df = df_shuffled[:split_index]
test_df = df_shuffled[split_index:]

#verificacion
print("\ntamaño data entrenamiento:", len(train_df))
print("\ntamaño data test:", len(test_df))



tamaño data entrenamiento: 4450

tamaño data test: 1113


In [ ]:
# c) Generar el vocabulario
vocabulary = set()

for tokens in train_df["tokens"]:
    for t in tokens:
        vocabulary.add(t)

print("\nTamaño del vocabulario:", len(vocabulary))


Tamaño del vocabulario: 7583


### Entrenamiento

In [ ]:
from collections import defaultdict

# Calculo de la probabilidad de spam y ham

num_spam = len(train_df[train_df["label"] == "spam"])
num_ham = len(train_df[train_df["label"] == "ham"])
total_msgs = len(train_df)

P_spam = num_spam / total_msgs
P_ham = num_ham / total_msgs

print("P(Spam):", P_spam)
print("P(Ham):", P_ham)


P(Spam): 0.13415730337078652
P(Ham): 0.8658426966292135


In [ ]:
# Conteo de palabras

spam_word_counts = defaultdict(int)
ham_word_counts = defaultdict(int)

spam_total_words = 0
ham_total_words = 0

for _, row in train_df.iterrows():
    if row["label"] == "spam":
        for word in row["tokens"]:
            spam_word_counts[word] += 1
            spam_total_words += 1
    else:
        for word in row["tokens"]:
            ham_word_counts[word] += 1
            ham_total_words += 1

In [ ]:
# Implementación de laplace con k=1

k = 1
V = len(vocabulary)

likelihood_spam = {}
likelihood_ham = {}

for word in vocabulary:
    likelihood_spam[word] = (
        spam_word_counts[word] + k
    ) / (spam_total_words + k * V)

    likelihood_ham[word] = (
        ham_word_counts[word] + k
    ) / (ham_total_words + k * V)


### Inferencia

In [ ]:
import math

# predicción de los mensajes
def predict(message):
    tokens = clean_text(message)

    log_spam = math.log(P_spam)
    log_ham = math.log(P_ham)

    for word in tokens:
        if word in vocabulary:
            log_spam += math.log(likelihood_spam[word])
            log_ham += math.log(likelihood_ham[word])

    return "spam" if log_spam > log_ham else "ham"


### Evaluación

In [ ]:
TP = FP = TN = FN = 0

for _, row in test_df.iterrows():
    pred = predict(row["message"])
    real = row["label"]

    if real == "spam" and pred == "spam":
        TP += 1
    elif real == "ham" and pred == "spam":
        FP += 1
    elif real == "ham" and pred == "ham":
        TN += 1
    elif real == "spam" and pred == "ham":
        FN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)

print("Matriz de Confusión")
print(f"TP: {TP}  FP: {FP}")
print(f"FN: {FN}  TN: {TN}")
print("\nAccuracy:", accuracy)


Matriz de Confusión
TP: 137  FP: 4
FN: 13  TN: 959

Accuracy: 0.9847259658580413
